<a href="https://colab.research.google.com/github/Omaar-X/Data_Mining/blob/main/Lab_Task_2__Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Dataset or Mount**

In [7]:
import pandas as pd

path="//content/drive/MyDrive/smoking_data/smoking.csv"

df=pd.read_csv(path)
df.head(5)



,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,12.9,1.0,0.7,18.0,19.0,27.0,Y,0,Y,0
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,12.7,1.0,0.6,22.0,19.0,18.0,Y,0,Y,0
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,15.8,1.0,1.0,21.0,16.0,22.0,Y,0,N,1
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,14.7,1.0,1.0,19.0,26.0,18.0,Y,0,Y,0
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,12.5,1.0,0.6,16.0,14.0,22.0,Y,0,N,0


In [8]:
import pandas as pd

# Please update this path with the correct location of your CSV file in Google Drive.
path = "/content/drive/MyDrive/path/to/your/smoking.csv"

try:
    df = pd.read_csv(path)
    print("DataFrame loaded successfully:")
    display(df.head(5))
except FileNotFoundError:
    print(f"Error: The file was not found at {path}. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: The file was not found at /content/drive/MyDrive/path/to/your/smoking.csv. Please check the path and try again.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

After mounting your drive, you can access your files at `/content/drive/MyDrive/`. Remember to replace the placeholder path in the previous code cell with the actual path to your CSV file.

**Label**

In [10]:
LABEL_COL = "smoking"
df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip()
df[LABEL_COL] = df[LABEL_COL].apply(lambda x: 1 if 'smok' in x.lower() else (0 if x.strip()=='0' or x.lower()=='non' or 'no' in x.lower() else 1))
df[LABEL_COL] = df[LABEL_COL].astype(int)
print(df[LABEL_COL].value_counts())


smoking
0    35237
1    20455
Name: count, dtype: int64


**Remove Constant & Non-Numeric Columns**

In [11]:
num_cols = df.select_dtypes(include=['number']).columns.tolist()
if LABEL_COL in num_cols: num_cols.remove(LABEL_COL)
X = df[num_cols].loc[:, (df[num_cols] != df[num_cols].iloc[0]).any()]  # drop constant
y = df[LABEL_COL].values
print("Numeric features:", X.shape[1])


Numeric features: 23


**Handle Inf / NaN Values**

In [12]:
import numpy as np
X.replace([np.inf, -np.inf], np.nan, inplace=True)
mask = ~X.isna().any(axis=1)
X = X[mask]
y = y[mask]
print("After dropping NaN rows:", X.shape)


After dropping NaN rows: (55692, 23)


Train/Validation/Test Split (Stratified)

In [13]:
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=42)
print(X_train.shape, X_val.shape, X_test.shape)


(35642, 23) (8911, 23) (11139, 23)


**Feature Selection (SelectKBest)**

In [14]:
from sklearn.feature_selection import SelectKBest, f_classif
k = min(20, X_train.shape[1])
selector = SelectKBest(f_classif, k=k)
selector.fit(X_train, y_train)
selected_cols = X_train.columns[selector.get_support()].tolist()
X_train = X_train[selected_cols].copy()
X_val   = X_val[selected_cols].copy()
X_test  = X_test[selected_cols].copy()
print("Selected:", selected_cols)


Selected: ['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'dental caries']


**StandardScaler (fit on train only)**

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)


**Apply SMOTE**

In [16]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_scaled, y_train)
print("After SMOTE:", X_train_bal.shape, np.bincount(y_train_bal))


After SMOTE: (45102, 20) [22551 22551]


**Dirichlet Non-IID Partition (10 Clients)**

In [17]:
import numpy as np
def dirichlet_partition(X, y, n_clients=10, alpha=0.5, seed=42):
    rng = np.random.default_rng(seed)
    labels = np.unique(y)
    idx = np.arange(len(y)); rng.shuffle(idx)
    Xs, ys = X[idx], y[idx]
    label_idx = {lab: np.where(ys==lab)[0] for lab in labels}
    client_idxs = [[] for _ in range(n_clients)]
    for lab in labels:
        li = label_idx[lab]
        props = rng.dirichlet([alpha]*n_clients)
        cuts = (np.cumsum(props) * len(li)).astype(int)[:-1]
        splits = np.split(li, cuts)
        for cid, s in enumerate(splits):
            client_idxs[cid].extend(s.tolist())
    splits = []
    for cid in range(n_clients):
        ids = np.array(client_idxs[cid], dtype=int)
        splits.append((Xs[ids], ys[ids]))
    return splits

clients = dirichlet_partition(X_train_bal, y_train_bal, n_clients=10, alpha=0.5, seed=42)
for i,(cx,cy) in enumerate(clients):
    print("Client", i, "->", cx.shape, np.bincount(cy))


Client 0 -> (4438, 20) [1188 3250]
Client 1 -> (1116, 20) [310 806]
Client 2 -> (2111, 20) [ 990 1121]
Client 3 -> (11366, 20) [ 1299 10067]
Client 4 -> (7171, 20) [6842  329]
Client 5 -> (8826, 20) [5929 2897]
Client 6 -> (331, 20) [167 164]
Client 7 -> (3372, 20) [1144 2228]
Client 8 -> (4144, 20) [2900 1244]
Client 9 -> (2227, 20) [1782  445]


**Define PyTorch MLP Model**

In [ ]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, in_dim, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
    def forward(self, x):
        return self.net(x)


**Local Training (FedAvg) — Client Side**

In [19]:
import torch, torch.optim as optim, torch.nn as nn
def local_train(global_state, client_X, client_y, epochs=10, lr=0.001, device='cpu'):
    model = MLP(client_X.shape[1]).to(device)
    model.load_state_dict(global_state)
    opt = optim.Adam(model.parameters(), lr=lr)
    xt = torch.tensor(client_X, dtype=torch.float32, device=device)
    yt = torch.tensor(client_y, dtype=torch.long, device=device)
    model.train()
    for e in range(epochs):
        opt.zero_grad()
        out = model(xt)
        loss = nn.CrossEntropyLoss()(out, yt)
        loss.backward()
        opt.step()
    return {k:v.detach().cpu() for k,v in model.state_dict().items()}


**Aggregation: FedAvg / FedProx / FedNova / FedDyn**

In [20]:
import copy
def avg_state_dicts(updates):
    avg = {}
    for k in updates[0].keys():
        avg[k] = sum([u[k] for u in updates]) / len(updates)
    return avg

def fed_nova(updates, local_steps):
    total = sum(local_steps)
    out = {}
    for k in updates[0].keys():
        acc=None
        for u,s in zip(updates, local_steps):
            acc = (u[k]*(s/total)) if acc is None else acc + u[k]*(s/total)
        out[k]=acc
    return out

def fed_dyn(updates, prev_global, alpha=0.1):
    avg = avg_state_dicts(updates)
    return {k: (1-alpha)*prev_global[k] + alpha*avg[k] for k in avg}


**FedProx Local Training with Prox-Term**

In [21]:
def prox_term(model, global_state, mu=0.001):
    prox = 0.0
    for name, p in model.named_parameters():
        prox += torch.sum((p - global_state[name].to(p.device))**2)
    return (mu/2.0) * prox

# in local loop:
# loss = ce_loss(out, yt) + prox_term(model, global_state, mu=mu_prox)


**Validation / Test Evaluation (Metrics)**

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
def evaluate_model(model, X_np, y_np, device='cpu'):
    model.eval()
    with torch.no_grad():
        xt = torch.tensor(X_np, dtype=torch.float32, device=device)
        logits = model(xt)
        probs = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        preds = logits.argmax(dim=1).cpu().numpy()
    acc = accuracy_score(y_np, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(y_np, preds, average='macro', zero_division=0)
    auc = float('nan')
    try:
        auc = roc_auc_score(y_np, probs)
    except:
        pass
    return acc, prec, rec, f1, auc


**Full Federated Loop**

In [24]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

# --- Device and Input Size ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = X_train_bal.shape[1]  # after preprocessing + SMOTE

# --- Define Model ---
class MLP(nn.Module):
    def __init__(self, in_dim, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# --- Local Training Function ---
def local_train(global_state, client_X, client_y, epochs=10, lr=0.001, device='cpu'):
    model = MLP(client_X.shape[1]).to(device)
    model.load_state_dict(global_state)
    opt = optim.Adam(model.parameters(), lr=lr)
    xt = torch.tensor(client_X, dtype=torch.float32, device=device)
    yt = torch.tensor(client_y, dtype=torch.long, device=device)
    model.train()
    for e in range(epochs):
        opt.zero_grad()
        out = model(xt)
        loss = nn.CrossEntropyLoss()(out, yt)
        loss.backward()
        opt.step()
    return {k:v.detach().cpu() for k,v in model.state_dict().items()}

# --- Aggregation Function (FedAvg) ---
def avg_state_dicts(updates):
    avg = {}
    for k in updates[0].keys():
        avg[k] = sum([u[k] for u in updates]) / len(updates)
    return avg

# --- Evaluation Function ---
def evaluate_model(model, X_np, y_np, device='cpu'):
    model.eval()
    with torch.no_grad():
        xt = torch.tensor(X_np, dtype=torch.float32, device=device)
        logits = model(xt)
        probs = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        preds = logits.argmax(dim=1).cpu().numpy()
    acc = accuracy_score(y_np, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(y_np, preds, average='macro', zero_division=0)
    auc = float('nan')
    try:
        auc = roc_auc_score(y_np, probs)
    except:
        pass
    return acc, prec, rec, f1, auc

# --- Federated Learning Loop ---
global_model = MLP(input_size).to(device)
global_state = copy.deepcopy(global_model.state_dict())

ROUNDS = 40
N_CLIENTS = 10
LOCAL_EPOCHS = 10
LR = 0.001

for rnd in range(ROUNDS):
    updates = []
    local_steps = []

    for cid, (cX, cY) in enumerate(clients):
        if len(cY) == 0:
            continue
        upd = local_train(global_state, cX, cY, epochs=LOCAL_EPOCHS, lr=LR, device=device)
        updates.append(upd)
        local_steps.append(len(cY)*LOCAL_EPOCHS)

    if len(updates) > 0:
        agg = avg_state_dicts(updates)
        global_model.load_state_dict(agg)
        global_state = copy.deepcopy(global_model.state_dict())

    # Optional: evaluate on validation after each round
    acc_v, prec_v, rec_v, f1_v, auc_v = evaluate_model(global_model, X_val_scaled, y_val, device=device)
    print(f"Round {rnd+1:02d} | Val ACC: {acc_v:.4f} | F1: {f1_v:.4f}")

# Final Test Evaluation
acc_t, prec_t, rec_t, f1_t, auc_t = evaluate_model(global_model, X_test_scaled, y_test, device=device)
print("\nFinal Test Metrics:")
print(f"Accuracy: {acc_t:.4f}, F1: {f1_t:.4f}, Precision: {prec_t:.4f}, Recall: {rec_t:.4f}, AUC: {auc_t}")


Round 01 | Val ACC: 0.6788 | F1: 0.6706
Round 02 | Val ACC: 0.6945 | F1: 0.6863
Round 03 | Val ACC: 0.6997 | F1: 0.6925
Round 04 | Val ACC: 0.7035 | F1: 0.6974
Round 05 | Val ACC: 0.7082 | F1: 0.7030
Round 06 | Val ACC: 0.7142 | F1: 0.7094
Round 07 | Val ACC: 0.7155 | F1: 0.7110
Round 08 | Val ACC: 0.7186 | F1: 0.7143
Round 09 | Val ACC: 0.7202 | F1: 0.7161
Round 10 | Val ACC: 0.7220 | F1: 0.7180
Round 11 | Val ACC: 0.7230 | F1: 0.7191
Round 12 | Val ACC: 0.7238 | F1: 0.7199
Round 13 | Val ACC: 0.7256 | F1: 0.7217
Round 14 | Val ACC: 0.7262 | F1: 0.7222
Round 15 | Val ACC: 0.7264 | F1: 0.7225
Round 16 | Val ACC: 0.7270 | F1: 0.7229
Round 17 | Val ACC: 0.7282 | F1: 0.7242
Round 18 | Val ACC: 0.7275 | F1: 0.7236
Round 19 | Val ACC: 0.7288 | F1: 0.7247
Round 20 | Val ACC: 0.7282 | F1: 0.7242
Round 21 | Val ACC: 0.7283 | F1: 0.7244
Round 22 | Val ACC: 0.7283 | F1: 0.7243
Round 23 | Val ACC: 0.7279 | F1: 0.7240
Round 24 | Val ACC: 0.7288 | F1: 0.7247
Round 25 | Val ACC: 0.7280 | F1: 0.7239


In [ ]:
# Step 0: Install required packages (if not already installed)
!pip install shap lime --quiet

# Step 1: Imports
from collections import defaultdict
import shap
from lime.lime_tabular import LimeTabularExplainer
import torch

# Step 2: Define a prediction function for SHAP and LIME
def model_predict(X):
    """
    X: numpy array
    Returns: probability predictions from the trained global_model
    """
    global_model.eval()
    with torch.no_grad():
        X_tensor = torch.tensor(X, dtype=torch.float32)
        logits = global_model(X_tensor)
        probs = torch.softmax(logits, dim=1).numpy()
    return probs

# ---------------- SHAP Explanation ----------------
# Use a smaller background dataset for speed
background = X_train_scaled[:100]

# Initialize SHAP Kernel Explainer
shap_explainer = shap.KernelExplainer(model_predict, background)

# Compute SHAP values for first 10 samples from test set
shap_values = shap_explainer.shap_values(X_test_scaled[:10], nsamples=100)

# Plot SHAP summary (bar plot)
shap.summary_plot(shap_values, X_test_scaled[:10], feature_names=selected_cols, plot_type="bar")

# ---------------- LIME Explanation ----------------
lime_explainer = LimeTabularExplainer(
    training_data=X_train_scaled,
    feature_names=selected_cols,
    class_names=["Non-Smoker", "Smoker"],
    discretize_continuous=True
)

# Explain a single instance (first test sample)
exp = lime_explainer.explain_instance(
    X_test_scaled[0],
    model_predict,
    num_features=10
)

# Show LIME explanation in notebook
exp.show_in_notebook()


In [ ]:
import pandas as pd

# Please update this path with the correct location of your CSV file in Google Drive.
path = "/content/drive/MyDrive/path/to/your/smoking.csv"

try:
    df = pd.read_csv(path)
    print("DataFrame loaded successfully:")
    display(df.head(5))
except FileNotFoundError:
    print(f"Error: The file was not found at {path}. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")